In [5]:
#arcpy to process LIDAR data    

import arcpy
from arcpy import env
import pandas as pd
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcpy.sa import *
import numpy

import sys
sys.path.append("..")

from src.features.create_rasters import *
from src.features.arcpy_scripts import *

In [8]:
#define variables

#town name
town_name = 'Revere'

#project geodatabase
env.workspace = r'K:\DataServices\Projects\Current_Projects\Climate_Change\MVP_MMC_CoolRoofs_MVP\ArcGIS\CoolRoofs_Analysis.gdb'

env.overwriteOutput = True

#folder that contains las data
las_folder = r"I:\Imagery\Eastern Mass Lidar\Revere Pilot"

#dataset file path
las_dataset = r"K:\DataServices\Datasets\MassGIS\LiDAR\Revere pilot\LAS Data\Revere pilot_LasDataset.lasd"

arcpy.env.outputCoordinateSystem = arcpy.SpatialReference("NAD 1983 StatePlane Massachusetts FIPS 2001 (Meters)")


In [5]:
#input variables

#DTM
ground_layer = town_name + '_ground_layer'
ground_code = [1]
ground_return_values = ['LAST', 'LAST_OF_MANY']
out_dtm_raster = town_name + '_dtm_surface'

#DSM
surface_layer = town_name + '_surface_layer'
surface_return_values = [1, 'FIRST_OF_MANY']
out_dsm_raster = town_name + '_building_dsm_surface'
building_code = [6]

#NDSM
in_raster1 = out_dsm_raster
in_raster2 = out_dtm_raster
out_ndsm_raster = town_name + '_ndsm_buildings'

#intensity
out_intensity_raster = town_name + '_intensity'


#ASPECT
out_aspect_raster_ndsm = town_name + '_aspect_buildings_ndsm'
out_aspect_raster_dsm = town_name + '_aspect_buildings_dsm'

#SLOPE
out_slope_raster_ndsm = town_name + '_slope_buildings_ndsm'

#try different sampling values
sampling_value = .75


## Roof Extraction

Drawing from [this tutorial](https://doc.arcgis.com/en/imagery/workflows/best-practices/las-dataset-to-create-dtm-and-dsm-rasters.htm).
- Also [this video](https://www.youtube.com/watch?v=2pThNwhJ418&t=727s)

Steps:
- Generate a Digital Terrain Model (DTM) (LAS Dataset to Raster)
- Generate a Digital Surface Model (DSM) (LAS Dataset to Raster)
- Generate the Normalized Digital Surface Model (nDSM) (raster calculator)
- Aspect and Slope of nDSM


In [9]:
## CREATE LAS DATASET ##

point_selection_method = 'CLOSEST_TO_CENTER'

arcpy.CreateLasDataset_management(input=las_folder,
                                  out_las_dataset=las_dataset,
                                  compute_stats='COMPUTE_STATS')
                                  

<Result 'K:\\DataServices\\Datasets\\MassGIS\\LiDAR\\Revere pilot\\LAS Data\\Revere pilot_LasDataset.lasd'>

In [10]:
## DIGITAL TERRAIN MODEL (DTM) ##
dtm_layer = arcpy.management.MakeLasDatasetLayer(in_las_dataset=las_dataset, 
                                                out_layer = ground_layer, 
                                                class_code = ground_code,
                                                return_values = ground_return_values)


arcpy.conversion.LasDatasetToRaster(in_las_dataset=dtm_layer, 
                                    out_raster=out_dtm_raster, 
                                    value_field='ELEVATION', 
                                    interpolation_type = 'BINNING AVERAGE LINEAR',
                                    sampling_type='CELLSIZE', 
                                    sampling_value=sampling_value)


## DIGITAL SURFACE MODEL (DSM) ##


dsm_layer = arcpy.management.MakeLasDatasetLayer(in_las_dataset=las_dataset, 
                                                out_layer = surface_layer, 
                                                class_code = building_code,
                                                return_values = surface_return_values)


arcpy.conversion.LasDatasetToRaster(in_las_dataset=dsm_layer, 
                                    out_raster=out_dsm_raster, 
                                    value_field='ELEVATION',
                                    interpolation_type = 'BINNING MAXIMUM NONE',
                                    sampling_type='CELLSIZE', 
                                    sampling_value=sampling_value)


## NORMALIZED DIGITAL SURFACE MODEL (NDSM) ##

# Excuate RasterCalculator(Minus) function
out_rc_minus_raster = RasterCalculator(rasters= [in_raster1, in_raster2], 
                                    input_names = ["x", "y"],
                                    expression="x-y")

# Save the output
#out_rc_minus_raster.save(out_ndsm_raster)

## ASPECT ## 

apect_ndsm = Aspect(in_raster=out_rc_minus_raster)
apect_ndsm.save(out_aspect_raster_ndsm)

## SLOPE ##
slope = Slope(in_raster=out_rc_minus_raster, 
                output_measurement='PERCENT_RISE')

slope.save(out_slope_raster_ndsm) #consider not saving if we go straight to the next step

## INTENSITY ## 
intensity_layer = arcpy.management.MakeLasDatasetLayer(in_las_dataset=las_dataset, 
                                                out_layer = surface_layer, 
                                                class_code = building_code,
                                                return_values = surface_return_values)


arcpy.conversion.LasDatasetToRaster(in_las_dataset=intensity_layer, 
                                    interpolation_type = 'BINNING AVERAGE NONE',
                                    out_raster=out_intensity_raster, 
                                    value_field='INTENSITY',
                                    sampling_type='CELLSIZE', 
                                    sampling_value=sampling_value)

<Result 'K:\\DataServices\\Projects\\Current_Projects\\Climate_Change\\MVP_MMC_CoolRoofs_MVP\\ArcGIS\\CoolRoofs_Analysis.gdb\\Revere_intensity'>

# Zonal Stats with slope and aspect

In [12]:
#now a bunch of zonal stats things


massgis_footprints = r'K:\DataServices\Datasets\MassGIS\Facilities_Structures\Building_Structures\Output\structures.gdb\STRUCTURES_POLY'
zonal_histogram_name = town_name + '_zonal_histogram'
zonal_stats_name = town_name + '_zonal_stats'
clipped_footprints = town_name + '_footprints' 
reclassified_slope_name = town_name + '_slope_reclassified'
zonal_stats_layer_name = town_name + '_zonal_stats_layer'


In [13]:
## CLIP FOOTPRINTS TO RASTER EXTENT ##

## RECLASSIFY SLOPE ##
if arcpy.Exists(clipped_footprints):
        arcpy.Delete_management(clipped_footprints)

if arcpy.Exists(town_name + '_footprints_with_info'):
        arcpy.Delete_management(town_name + '_footprints_with_info')

#first get extent
pnt_array = arcpy.Array()
extent = arcpy.Raster(out_slope_raster_ndsm).extent
pnt_array.add(extent.lowerLeft)
pnt_array.add(extent.lowerRight)
pnt_array.add(extent.upperRight)
pnt_array.add(extent.upperLeft)

#polygon of extent
clip_poly = arcpy.Polygon(pnt_array)

#now clip
arcpy.analysis.Clip(massgis_footprints, clip_poly, clipped_footprints)


#first build attribute table
slope_reclassified = Reclassify(in_raster=out_slope_raster_ndsm, 
                        reclass_field="VALUE", 
                        remap="0 15 1; 15 20 2;20 30 3;30 40 4;40 50 5;50 60 6;60 10000 7",  #add a variable for this
                        missing_values="NODATA")

## ZONAL HISTOGRAM ## 
zonal_histogram = ZonalHistogram(
                        in_zone_data=clipped_footprints,
                        zone_field="STRUCT_ID",
                        in_value_raster=slope_reclassified,
                        out_table=zonal_histogram_name,
                        out_graph="",
                        zones_as_rows="ZONES_AS_ROWS"
                    )

#rename fields in histogram (automatically churns out CLASS_1, etc, but we want to specify slope)
fieldList = arcpy.ListFields(zonal_histogram)

for field in fieldList:
    if field.name.split('_')[0] == 'CLASS':
        arcpy.AlterField_management(in_table=zonal_histogram, 
                                    field=field.name, 
                                    new_field_alias=('SLOPE_' + field.name.split('_')[1]),
                                    new_field_name=('SLOPE_' + field.name.split('_')[1]))
    else:
        pass


#need to add a field to make a join possible with zonal histogram
new_field = "join_id"
arcpy.management.CalculateField(
            in_table=clipped_footprints,
            field=new_field,
            expression="'STRUC_'+ !STRUCT_ID!",
            expression_type="PYTHON3",
            code_block="",
            field_type="TEXT",
            enforce_domains="NO_ENFORCE_DOMAINS"
        )

## NOW JOIN ZONAL STATS + HISTOGRAM DATA TO STRUCTURES ##
arcpy.management.JoinField(
                            in_data=clipped_footprints,
                            in_field=new_field,
                            join_table=zonal_histogram,
                            join_field="LABEL",
                            fields=['SLOPE_1']
                        )


#SELECT ONLY NONNULL
arcpy.analysis.Select(
            in_features=clipped_footprints,
            out_feature_class=town_name + '_footprints_with_info',
            where_clause="SLOPE_1 IS NOT NULL",
        )

## ZONAL STATISTICS AS TABLE ##
with arcpy.EnvManager(snapRaster=slope_reclassified, extent=extent, cellSize=out_slope_raster_ndsm):
    ZonalStatisticsAsTable(
                    in_zone_data=town_name + '_footprints_with_info',
                    zone_field="STRUCT_ID",
                    in_value_raster=slope_reclassified,
                    out_table= zonal_stats_name,
                    ignore_nodata="DATA",
                    statistics_type="MAJORITY_VALUE_COUNT_PERCENT"
                    )

#ADD A FIRST PASS FLAT ROOF FIELD
field_name = 'flat_roof'
expression = "calc(!MAJORITY!)"
code_block = """
def calc(field1):
    if (field1 == 1):
        return 1
    else:
        return 0"""


arcpy.management.CalculateField(
                            in_table=zonal_stats_name,
                            field=field_name,
                            expression=expression,
                            expression_type="PYTHON3",
                            code_block=code_block,
                            field_type="DOUBLE",
                            enforce_domains="NO_ENFORCE_DOMAINS"
                        )

arcpy.management.JoinField(
                            in_data=town_name + '_footprints_with_info',
                            in_field='STRUCT_ID',
                            join_table=zonal_stats_name,
                            join_field="STRUCT_ID",
                            fields=['MAJORITY', 'MAJORITY_PERCENT', 'flat_roof']
                        )

## NOW DO INTENSITY ## 
intensity_reclassified = reclassify_by_quantiles(out_intensity_raster, 8)


                                    
## ZONAL STATISTICS AS TABLE ##
with arcpy.EnvManager(snapRaster=intensity_reclassified, extent=extent, cellSize=intensity_reclassified):
    ZonalStatisticsAsTable(
                    in_zone_data=town_name + '_footprints_with_info',
                    zone_field="STRUCT_ID",
                    in_value_raster=intensity_reclassified,
                    out_table= zonal_stats_name,
                    ignore_nodata="DATA",
                    statistics_type="MAJORITY_VALUE_COUNT_PERCENT"
                    )

arcpy.AlterField_management(in_table=zonal_stats_name, 
                                    field='MAJORITY', 
                                    new_field_alias='Int_Maj',
                                    new_field_name='Int_Maj')

                                    
arcpy.AlterField_management(in_table=zonal_stats_name, 
                                    field='MAJORITY_PERCENT', 
                                    new_field_alias='Int_MajP',
                                    new_field_name='Int_MajP')

arcpy.management.JoinField(
                            in_data=town_name + '_footprints_with_info',
                            in_field='STRUCT_ID',
                            join_table=zonal_stats_name,
                            join_field="STRUCT_ID",
                            fields=['Int_Maj', 'Int_MajP']
                        )


arcpy.Delete_management(zonal_histogram_name)
arcpy.Delete_management(zonal_stats_name)
arcpy.Delete_management(clipped_footprints)
arcpy.Delete_management(zonal_stats_layer_name)
arcpy.Delete_management(clipped_footprints)


<Result 'true'>

In [ ]:
## NDSM ##

In [ ]:
arcpy.management.BuildLasDatasetPyramid(in_las_dataset=las_dataset,
                                 point_selection_method=point_selection_method)